In [16]:
from typing import Tuple, Dict
import torch

import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch import Tensor
from torchvision.datasets import CIFAR10

In [17]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16 * 5* 5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x:Tensor) -> Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [18]:
DATA_ROOT = "E:/data"

def load_data() -> Tuple[
    torch.utils.data.DataLoader,
    torch.utils.data.DataLoader,
    Dict
]:
    transform = transforms.Compose(
        [   
            transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
        ]
    )
    trainset = CIFAR10(DATA_ROOT,train = True,download=True,transform=transform)
    
    trainloader = torch.utils.data.DataLoader(trainset,batch_size = 32,shuffle = True)
    
    testset = CIFAR10(DATA_ROOT,train = False,download=True,transform=transform)
    
    testloader = torch.utils.data.DataLoader(testset,batch_size = 32,shuffle = False)
    
    num_examples = {
        "trainset":len(trainset),
        "testset":len(testset)
    }
    return trainloader,testloader,num_examples

In [19]:
def train(
    net: Net,
    trainloader: torch.utils.data.DataLoader,
    epochs: int,
    device: torch.device,
) -> None:
    """Train the network."""
    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    print(f"Training {epochs} epoch(s) w/ {len(trainloader)} batches each")
    # Train the network
    for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            images, labels = data[0].to(device), data[1].to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
            if i % 100 == 99:  # print every 100 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1,i + 1,running_loss / 2000))
                running_loss = 0.0

In [20]:
def test(net:Net,testloader:torch.utils.data.DataLoader,device:torch.device)->Tuple[float,float]:
    criterion = nn.CrossEntropyLoss()
    correct = 0
    total = 0
    loss = 0.0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return loss, accuracy

In [21]:
import datetime
def main():
    now_time = datetime.datetime.now()
    DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Centralized PyTorch training")
    print("Load data")
    trainloader, testloader, _ = load_data()
    print("Start training")
    net=Net().to(DEVICE)
    train(net=net, trainloader=trainloader, epochs=2, device=DEVICE)
    print("Evaluate model")
    loss, accuracy = test(net=net, testloader=testloader, device=DEVICE)
    print("Loss: ", loss)
    print("Accuracy: ", accuracy)
    finishtime = datetime.datetime.now()
    print(finishtime - now_time)
if __name__ == "__main__":
    main()

Centralized PyTorch training
Load data
Files already downloaded and verified
Files already downloaded and verified
Start training
Training 2 epoch(s) w/ 1563 batches each
[1,   100] loss: 0.115
[1,   200] loss: 0.115
[1,   300] loss: 0.115
[1,   400] loss: 0.115
[1,   500] loss: 0.115
[1,   600] loss: 0.115
[1,   700] loss: 0.115
[1,   800] loss: 0.115
[1,   900] loss: 0.115
[1,  1000] loss: 0.114
[1,  1100] loss: 0.114
[1,  1200] loss: 0.113
[1,  1300] loss: 0.112
[1,  1400] loss: 0.110
[1,  1500] loss: 0.109
[2,   100] loss: 0.105
[2,   200] loss: 0.104
[2,   300] loss: 0.102
[2,   400] loss: 0.102
[2,   500] loss: 0.098
[2,   600] loss: 0.098
[2,   700] loss: 0.097
[2,   800] loss: 0.096
[2,   900] loss: 0.093
[2,  1000] loss: 0.093
[2,  1100] loss: 0.092
[2,  1200] loss: 0.092
[2,  1300] loss: 0.091
[2,  1400] loss: 0.089
[2,  1500] loss: 0.088
Evaluate model
Loss:  542.5961790084839
Accuracy:  0.3703
0:00:35.198373
